# Adding Suggestions
This notebook will walk you through adding suggestions to LightTag throw the API. 

Conceptually, Suggestions come from a Model and a Model belongs to a Schema. Thus in order to add Suggestions you must register a Model under the relevant Schema and then populate the model with actual suggestions. 

To avoid confusion, while you are registering a Model, LightTag will not run your model, instead, it is up to you to generate the suggestions and upload them. Here we'll see how to do that

## The Steps
1. Retreiving the Examples we wish to generate suggestions on
2. Retreiveing the Schema we wish to define a Model for
3. Creating a model and suggestions
4. Uploading the suggestions and Model
5. Assigning a model to a task definition

## Step 0 - Basic setup

In [5]:
import requests
import json
import pandas as pd
from pprint import pprint
from requests.auth import HTTPBasicAuth

LIGHTTAG_DOMAIN = 'demo'  #should be your lighttag domain
SERVER = 'https://{domain}.lighttag.io/api/'.format(domain=LIGHTTAG_DOMAIN) 
API_BASE = SERVER +'v1/'
LT_USERNAME = "MY_USER" # Username of manager user
LT_PASSWORD = "MY_PSWD" #password of manager user

response = requests.post(SERVER +"auth/token/login/",
              json={"username":LT_USERNAME,"password":LT_PASSWORD}
             )
response

<Response [200]>

Once we log in, LightTag will return a token we can use for further calls. We'll put that in dict that we will pass as header

In [6]:
token = response.json()['key']
headers={'Authorization': 'Token {token}'.format(token=token)}
session = requests.session()
session.headers = headers

## Step 1 Getting the data examples we wish to annotate
To generate suggestions we will need data. Particularly we'll need both all of the examples in a dataset as well as the tags from a particular schema. 
While you could gather each of these seperatly, LightTag has a data download view which already returns both of these combined. 



### Retreiv the slug or url of the dataset with the examples you wish to add suggestions to
You can query the datasets endpoint. Remember, you are operating under the default project, so you should query 


projects/**default**/datasets/

In [7]:
datasets = session.get(API_BASE+'projects/default/datasets/').json()
pd.DataFrame(datasets) 
#It's convenient to display the results in a dataframe

,aggregation_field,archived,content_field,id,id_field,name,order_field,project_id,slug,upload_status,url
0,chapter,False,content,2a6e0c06-017d-473c-b3d5-bbb24c28b71c,None,Bible,verse,2753ca38-69d9-4c96-9d31-df6d4069b027,bible,done,https://demo.lighttag.io/api/v1/projects/defau...
1,None,False,FIELD2,1bf62f49-d88e-4bf0-9178-79012ce355c4,FIELD1,Drilling Comments Small,None,2753ca38-69d9-4c96-9d31-df6d4069b027,drilling-comments-small,done,https://demo.lighttag.io/api/v1/projects/defau...
2,None,False,content,1709c71b-2092-4d6b-800c-1341cae915ec,None,Some new dataset,None,2753ca38-69d9-4c96-9d31-df6d4069b027,some-new-dataset,done,https://demo.lighttag.io/api/v1/projects/defau...
3,None,False,data,f620a31a-c907-43cb-9f66-6642736d4b4f,id,Jabberwocky,None,2753ca38-69d9-4c96-9d31-df6d4069b027,jabberwocky,done,https://demo.lighttag.io/api/v1/projects/defau...
4,chapter,False,content,e04f9a51-acb4-44a4-b633-0d03fc6f7af4,None,Test,verse,2753ca38-69d9-4c96-9d31-df6d4069b027,test,done,https://demo.lighttag.io/api/v1/projects/defau...
5,None,False,content,24408e1f-dfe8-4d37-8f42-7192de1956e5,None,hebrew medical,None,2753ca38-69d9-4c96-9d31-df6d4069b027,hebrew-medical,done,https://demo.lighttag.io/api/v1/projects/defau...
6,date,False,text,045896b7-6e3e-427f-b920-7241da0088ae,None,Trump Tweets,time,2753ca38-69d9-4c96-9d31-df6d4069b027,trump-tweets,done,https://demo.lighttag.io/api/v1/projects/defau...
7,None,False,Message,ff31041a-69ac-4641-8f7f-6cad8bd257b7,None,öäå test,None,2753ca38-69d9-4c96-9d31-df6d4069b027,oaa-test,done,https://demo.lighttag.io/api/v1/projects/defau...
8,None,False,Message,4ddb421d-28b0-42a5-bb5c-54509b35d27b,None,format test,None,2753ca38-69d9-4c96-9d31-df6d4069b027,format-test,done,https://demo.lighttag.io/api/v1/projects/defau...
9,chapter,False,content,f44b27f2-e84b-441b-b6e9-ff7049111b6f,None,zxzx,verse,2753ca38-69d9-4c96-9d31-df6d4069b027,zxzx,done,https://demo.lighttag.io/api/v1/projects/defau...


### Retreive the examples from that dataset by 
In this example we what a Dataset named Test Set with a slug test-set. We'll pull all of the examples via the examples endpoint
projects/default/datasets/**test-set**/examples/

In [8]:

examples =session.get(API_BASE+'projects/default/datasets/test-set/examples/').json()

In [10]:
examples

[]

We could equivalently do this by usig the URL field on the json returned from the datasets endpoint

In [9]:
bible_dataset = next(filter(lambda x:x['slug']=="test-set",datasets))

In [10]:
examples =session.get(bible_dataset['url']+'examples/').json()

In [11]:
examples[0]

{'aggregation_value': '2017-03-19',
 'content': '#ICYMI: Weekly Address \n➡️https://t.co/ckVx2zgA1x https://t.co/dTGZLvlsGv',
 'dataset': '92ea39c8-1d7d-47eb-b6d3-25ceebc6ba33',
 'id': '734c1e07-3648-44c2-903c-df7ba777c7e5',
 'metadata': {'created_at': 'Sun Mar 19 20:20:22 +0000 2017',
  'date': '2017-03-19',
  'favorite_count': 36626,
  'id_str': '843557782666317826',
  'in_reply_to_user_id_str': None,
  'is_retweet': False,
  'retweet_count': 7472,
  'source': 'Twitter for iPhone',
  'time': 1489954822000000000}}

## Step 2 Retreive the Schema and Tags we want to create suggestions for


In [12]:
schemas =session.get(API_BASE+'projects/default/schemas/').json()
pd.DataFrame(schemas)

,id,name,slug,url
0,3b73c2b2-73aa-4c42-a265-a0c459abd295,Classifications and tags Schema,classifications-and-tags-schema,http://localhost:8000/api/v1/projects/default/...
1,fbe6fd1d-bb59-4909-8a88-81772cc0d996,Entity Tags only,entity-tags-only,http://localhost:8000/api/v1/projects/default/...
2,9dcf0091-d82c-4292-a09d-eaff062fc1c4,Trump Insult Classification,trump-insult-classification,http://localhost:8000/api/v1/projects/default/...


In [45]:
schema

{'id': 'fbe6fd1d-bb59-4909-8a88-81772cc0d996',
 'name': 'Entity Tags only',
 'slug': 'entity-tags-only',
 'url': 'http://localhost:8000/api/v1/projects/default/schemas/entity-tags-only/'}

In [14]:
schema = schemas[1]
tags = session.get(schema['url']+'tags/').json()
pd.DataFrame(tags)

,description,id,name
0,Republicans/Democrates etc,0b4e9d4d-96b2-4288-bd52-e269a505cce3,Politcal Group
1,CNN/Fox etc.,14fd0d03-006b-4372-823e-41e32642e3b4,Media organization
2,A word or phrase that is unsulting.,2b162d01-637d-44b6-acf6-77cb8081608f,Insult
3,A political issue the President is discussing.,08728ae4-61c9-43ea-86f4-ffbc5289c5ae,Issue
4,A physical place. For example the White House.,6d331788-d937-4a11-ac38-4d653e56ddbd,Place
5,The proper name of a person (John is a person....,038de1c3-143a-40fc-a14c-627920311055,Person


### Pro Tip 
To upload suggestions we'll need to use the tag ids, which can be cumbersome. It is often convenient to make a map
drom the tag name to the id like so : 

In [15]:
tagMap = {tag["name"]:tag["id"] for tag in tags}

In [16]:
tagMap

{'Insult': '2b162d01-637d-44b6-acf6-77cb8081608f',
 'Issue': '08728ae4-61c9-43ea-86f4-ffbc5289c5ae',
 'Media organization': '14fd0d03-006b-4372-823e-41e32642e3b4',
 'Person': '038de1c3-143a-40fc-a14c-627920311055',
 'Place': '6d331788-d937-4a11-ac38-4d653e56ddbd',
 'Politcal Group': '0b4e9d4d-96b2-4288-bd52-e269a505cce3'}

## Step 3: Create your suggestions
How you create a suggestion is up to you. Youa re free to use a dicitonary, regex, neural network or whatever else
is availble. 
In this example we'll use the [flashtext](https://github.com/vi3k6i5/flashtext) library to suggest the following




In [18]:
from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()
keyword_processor.add_keyword("Trump",tagMap["Person"])
keyword_processor.add_keyword("Wall",tagMap["Issue"])
keyword_processor.add_keyword("CNN",tagMap["Media organization"])
keyword_processor.add_keyword("Dumb",tagMap["Insult"])

keyword_processor.add_keyword("Russia",tagMap["Place"])
keyword_processor.add_keyword("Democrats",tagMap["Politcal Group"])


True

In [20]:
keyword_processor.extract_keywords(" Democrats don't want Trump to go to Russia. Dumb! ",
                                   span_info=True)

[('0b4e9d4d-96b2-4288-bd52-e269a505cce3', 1, 10),
 ('038de1c3-143a-40fc-a14c-627920311055', 22, 27),
 ('6d331788-d937-4a11-ac38-4d653e56ddbd', 37, 43),
 ('2b162d01-637d-44b6-acf6-77cb8081608f', 45, 49)]

### Step 3.1: Iterate over your examples and make a  list of suggestions


In [21]:
suggestions = []
for example in examples: #(Notice the text is located in the content field)
    #For every suggestion that comes out of our model (in this case, flashtext)
    for tag_id,start,end in keyword_processor.extract_keywords(example['content'],span_info=True):
        suggestion= { #Create a suggestion
            "example_id":example['id'], #That refers to a particular example
            "tag_id":tag_id, #and applies a particular tag
            "start":start, #Which starts somewhere in the example
            "end":end # And ends somewhere in the example
        }
        suggestions.append(suggestion)

### Step 3.2: Define a model

In [24]:
model_metadata= { # Define any metadata you'd like to store about the model
    "defined_by": "LightTag",
    "comments": "An example model made with Flashtext"
}
data = {
    "model":{
        "name":"demo_suggestions",  #Give the model a name
        "metadata": model_metadata # Provide metadata (optional)
    },
    "suggestions":suggestions #Attatch the suggestions you made before
}

## Step 4 Upload your model and Suggestions 

In [25]:
schema
resp =session.post(schema['url']+'models/bulk/',
                    json=data,
             )
resp.status_code

201

In [26]:
resp.json()

{'id': '94fc9df1-4dcf-4542-bb0f-e2b74a3d2ead',
 'metadata': {'comments': 'An example model made with Flashtext',
  'defined_by': 'LightTag'},
 'name': 'demo_suggestions',
 'slug': 'demo_suggestions',
 'url': 'http://localhost:8000/api/v1/projects/default/schemas/entity-tags-only/models/demo_suggestions/'}

## Step 5 Assign you suggestion model to a task
Once we have a model, we need to tell LightTag to show its results to our annotators. Suggestions are shown with the rest of the work, and so we assign one or more models to an individual task. 
In this example we'll assign  two models to our task. The one we defined and the LightTag default model

### Get the model ids for the schema we defined this model on

In [28]:
schema

{'id': 'fbe6fd1d-bb59-4909-8a88-81772cc0d996',
 'name': 'Entity Tags only',
 'slug': 'entity-tags-only',
 'url': 'http://localhost:8000/api/v1/projects/default/schemas/entity-tags-only/'}

In [32]:
models = session.get(schema['url']+'models/').json()
pprint(models)

[{'id': '94fc9df1-4dcf-4542-bb0f-e2b74a3d2ead',
  'metadata': {'comments': 'An example model made with Flashtext',
               'defined_by': 'LightTag'},
  'name': 'demo_suggestions',
  'slug': 'demo_suggestions',
  'url': 'http://localhost:8000/api/v1/projects/default/schemas/entity-tags-only/models/demo_suggestions/'},
 {'id': '78352a55-6a97-4973-a00d-77c031fa15e8',
  'metadata': {},
  'name': 'Entity Tags only ad hoc dict',
  'slug': 'entity-tags-only-ad-hoc-dict',
  'url': 'http://localhost:8000/api/v1/projects/default/schemas/entity-tags-only/models/entity-tags-only-ad-hoc-dict/'}]


### PUT the model ids into the suggestion field of the task


In [44]:
modelIds = {"models": [x['id'] for x in models]}

session.put(API_BASE+'projects/default/task_definitions/explore-tags-only/',json=modelIds).json()

[{'active': True,
  'allow_suggestions': True,
  'annotators_per_example': 2,
  'dataset_id': 'ee688ba1-b0d3-4fdb-9355-d7924e3875e4',
  'dataset_slug': 'exploratory-dataset',
  'guidelines': None,
  'id': '9bab798d-c8db-43e8-9d4e-bf893a2f83f0',
  'name': 'Explore tags and classes',
  'priority': 1,
  'project_id': 'e488e45a-564a-4b93-8beb-f7aa4c73ea97',
  'schema_id': '3b73c2b2-73aa-4c42-a265-a0c459abd295',
  'schema_slug': 'classifications-and-tags-schema',
  'slug': 'explore-tags-and-classes',
  'teams': [{'description': 'People who are very knowledgable about twitter',
    'id': 'd2648770-7886-4c41-ad7d-b1f6f713ffbd',
    'members': [{'id': 2, 'username': 'Bob'},
     {'id': 3, 'username': 'Hanz'},
     {'id': 1, 'username': 'demo'}],
    'name': 'Experts',
    'slug': 'experts',
    'url': 'http://localhost:8000/api/v1/projects/default/teams/experts/'}],
  'url': 'http://localhost:8000/api/v1/projects/default/task_definitions/explore-tags-and-classes/'},
 {'active': True,
  'allow_